In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from numpy.linalg import inv

from astropy.table import Table, Column, vstack, hstack, unique, SortedArray,SCEngine
import astropy.units as u

from astropy.io import fits, ascii 

import glob
import os

import numpy
from scipy.signal import medfilt
from scipy.interpolate import interp1d
from numba import njit
import tqdm
from pandas import DataFrame 

In [ ]:
# read in high S/N table 
# loop of over the rows 
# Night and Date to make spectra file name: PETAL_LOC, NIGHT, TILEID
# load hdul
# cut r camera, "r_wavelength" 
# Grab wavelength 
# Grab corresponding spectrum 
# FIBER #

In [2]:
highSN = Table.read("/Volumes/GoogleDrive/My Drive/HighS_N.fits")   # reads in table from previous code

PETAL = (highSN['PETAL_LOC'].data)  # load each column as an array
NIGHT = (highSN['NIGHT'].data)  # load each column as an array
TILEID = (highSN['TILEID'].data)  # load each column as an array

combined = np.vstack((PETAL, TILEID,NIGHT)).T  # combines each element of each array together 
print(combined)

[[       3    63075 20200305]
 [       1    63075 20200305]
 [       6    63075 20200305]
 ...
 [       7    63075 20200219]
 [       5    63159 20200314]
 [       8    63159 20200314]]


In [3]:
tileid = []  # creates an empty array
for row in highSN:
    file = str(row['PETAL_LOC']) + '-' + str(row['TILEID']) + '-' + str(row['NIGHT'])  # goes through each element of the array and grabs the wanted elements of each one. This then combines them in the right format 
    file_tileid = str(row['TILEID']) + '/' + str(row['NIGHT']) + '/coadd-' + file  # this grabs the necessary element of each array and combines them to make part of our path in the next cell
    tileid.append(file_tileid)  # appends the file created above to them empty array
#     print(file)

In [4]:
file = ['/Volumes/GoogleDrive/My Drive/andes (1)/tiles/' + x +'.fits' for x in tileid]  # this combines all of the elements grabbed above to make a filepath


In [ ]:
# for x,y in zip(list1,list2):

In [5]:
for i in range(2843,3843):

    hdul = fits.open(file[i])  # opens the fit data that belongs to the i sub_file and gets the information from that file

    r_wave = (hdul['R_WAVELENGTH'].data)  # Takes the chosen row of the hdul file
    r_flux = (hdul['R_FLUX'].data)  # Takes the chosen row of the hdul file
    r_ivar = (hdul['R_IVAR'].data)  # Takes the chosen row of the hdul file
    
    FIBER = (highSN['FIBER'].data)   # Takes the chosen row of the hdul file

    fibermap = hdul['FIBERMAP'].data  # Takes the chosen row of the hdul file
    fibers = fibermap['FIBER']   

#     print(FIBER[i])     # prints each element of FIBER 
    index = (np.where(np.in1d(fibers, FIBER[i])))   # prints which index is where fibers and FIBER matches
#     print(fibers[np.where(np.in1d(fibers, FIBER[i]))])  # plugs in the index to make sure this is where the number matches 
    index_ = list(index[0])  # converts the first element of the tuple created and converts it to a list.
#     print(index_[0])  # prints the first element of the list 
    
    
    rflux = r_flux[index_[0],:]   # plugs in the index found above and finds the matching spectrum 
    rivar = r_ivar[index_[0],:]   # plugs in the index found above and finds the matching spectrum 
    rwave = r_wave
    
    np.savez('/Volumes/GoogleDrive/My Drive/rflux.npz', rflux = rflux, overwrite = True)   # saves the multiple arrays to one file 
    np.savez('/Volumes/GoogleDrive/My Drive/rivar.npz', rivar = rivar, overwrite = True)
    np.savez('/Volumes/GoogleDrive/My Drive/rwave.npz', rwave = rwave, overwrite = True)
    
#     plt.title('Spectrum', fontsize = 15)  #  places a title and sets font size
#     plt.xlabel('Wavelength', fontsize = 15)   # places a label on the x axis and sets font size 
#     plt.ylabel('$\\mathrm{flux\\,[10^{-17}\\, erg \\, cm^{-2} \\, s^{-1} \\, \\AA^{-1}] }$', fontsize = 15) # places a label on the y axis and sets font size

#     plt.plot(r_wave, rflux)  # plots the lists we just created using a function from matplotlib.pyplot. This plots both the x and y lists.
#     plt.show() 

In [ ]:
#  Check the IVAR array for 0 or negative values
#  Set  the flux = `np.nan` for any pixels that have that IVAR <=0

In [30]:
from scipy.signal import medfilt
from scipy.interpolate import interp1d
from numba import njit
import tqdm
import matplotlib.pyplot as plt


flux = np.load('/Volumes/GoogleDrive/My Drive/rflux.npz', allow_pickle=True)# Load the spectra (expecting a list of spectra, length of the list is the number of objects, 
waves = np.load('/Volumes/GoogleDrive/My Drive/rwave.npz', allow_pickle=True)# Load the corresponding wavelength grid for each spectrum (also a list)
ivar = np.load('/Volumes/GoogleDrive/My Drive/rivar.npz', allow_pickle=True)

s = 3800
e = 7400
# common_wave = numpy.exp(numpy.linspace(numpy.log(s), numpy.log(e), 4200)) # Define the wavelength grid you would like to work with

flux = flux['rflux']
waves = waves['rwave']
ivar = ivar['rivar']

badpix = ivar <= 0
flux[badpix] = np.nan

print(flux)

nof_objects = len(flux)

@njit
def remove_outliers_and_nans(flux, flux_): 
    ###
    
#     Use nearby pixels to remove 3 sigma outlires and nans 
    
    ##
    nof_features = flux.size
    d = 5
    for f in range(d,nof_features-d):
        val_flux = flux[f]
        leave_out = numpy.concatenate((flux[f-d:f],flux[f+1:f+d]))
        leave_out_mean = numpy.nanmean(leave_out)
        leave_out_std = numpy.nanstd(leave_out)

        if abs(val_flux - leave_out_mean) > 3*leave_out_std:
            flux_[f] = leave_out_mean

        d_ = d
        while not numpy.isfinite(flux_[f]):
            val_flux = flux[f]
            d_ = d_ + 1
            leave_out = numpy.concatenate((flux[f-d_:f],flux[f+1:f+d_]))
            leave_out_mean = numpy.nanmean(leave_out)
            flux_[f] = leave_out_mean

    return flux_

specs_same_grid = []
for wave, spec in zip(waves, flux):
    specs_same_grid += [numpy.interp(wave, flux)]
    
specs_same_grid = numpy.array(specs_same_grid)
for i in range(nof_objects):
    flux = flux.copy()
    flux_ = flux.copy()
    # remove outliers and nans
    specs_final[i] = remove_outliers_and_nans(flux, flux_)
    # 5 pixel median filter (to remove some of the noise)
    specs_final[i] = medfilt(specs_final[i], 5)

# specs_same_grid = []
# for wave, spec in zip(waves, specs):
#     specs_same_grid += [numpy.interp(common_wave, wave, spec)]
# specs_same_grid = numpy.array(specs_same_grid)



# specs_final = numpy.zeros(specs_same_grid.shape)
# for i in range(nof_objects):
#     s = specs_same_grid[i].copy()
#     s_ = s.copy()
#     # remove outliers and nans
#     specs_final[i] = remove_outliers_and_nans(s, s_)
#     # 5 pixel median filter (to remove some of the noise)
#     specs_final[i] = medfilt(specs_final[i], 5)

[3231.3555 3915.4768 4149.006  4223.501  4091.3186 4027.7656 3704.3708
 4500.7056 4260.331  3842.5964 4225.3174 4119.8506 3942.4368 4118.08
 4418.948  3908.3147 4252.978  3900.7166 4178.6826 4210.8257 3896.883
 4265.458  4104.7207 4148.7646 4077.026  4188.77   4113.0024 4166.0205
 3924.6562 3920.2302 3898.8967 3893.431  4000.9026 3699.5115 3851.5542
 3963.1167 3975.1147 4017.1743 3904.2192 4016.8894 4063.0276 3913.5576
 3946.9807 4028.0637 4006.8662 3862.0903 3795.69   3999.597  4009.4072
 3921.952  3924.6096 4120.185  3883.2444 3881.737  4111.4414 3888.4216
 4009.4275 3741.055  3852.6938 3875.1938 3837.1343 3820.6184 3937.7256
 3867.7546 4079.2808 4086.0327 3991.7134 3955.8452 3857.3455 3887.1226
 3936.9827 3846.668  3908.7073 3903.5776 3845.258  3805.4407 3851.5452
 3919.82   3954.954  3962.8103 3890.4797 3864.293  3865.4543 3874.0393
 3903.508  3995.3325 4072.9885 3892.4666 3978.897  3876.5354 3838.0854
 3844.4856 3918.4976 3990.2246 3904.735  3806.123  3921.9294 3852.3965
 3776.286

TypeError: _interp_dispatcher() missing 1 required positional argument: 'fp'

In [ ]:
plt.figure(figsize = (15,7))
idx = numpy.random.choice(specs.shape[0])
plt.rcParams['figure.figsize'] = 10, 4
plt.figure()
plt.title('Original')
plt.step(waves[idx], specs[idx], "k")
plt.xlabel("observed wavelength")
plt.ylabel("$\\mathrm{flux\\,[10^{-17}\\, erg \\, cm^{-2} \\, s^{-1} \\, \\AA^{-1}] }$")
plt.tight_layout()

plt.figure()
plt.title('Noise removed, common grid')
plt.step(common_wave, specs_final[idx], "k")
plt.xlabel("observed wavelength")
plt.ylabel("$\\mathrm{flux\\,[10^{-17}\\, erg \\, cm^{-2} \\, s^{-1} \\, \\AA^{-1}] }$")
plt.tight_layout()

In [ ]:
import umap
fit = umap.UMAP()

In [ ]:
em = fit.fit_transform(specs_final)
x = em[:,0]
y = em[:,1]

In [ ]:
plt.figure(figsize = (8,7))
plt.scatter(x, y)
plt.show()